In [1]:
import pandas as pd
import numpy as np
import pickle 
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Loading Inputs

In [2]:
# for some weird reason required in Jupyter Lab when unpickling model, scaler
import sys
sys.path.append('..')

In [3]:
model = pickle.load(open('../data/06_models/model.pkl/2020-10-09T15.53.34.176Z/model.pkl', 'rb'))
model['pass 1']

In [4]:
model

{'pass 1': <src.utils.modeling.ForecastingModel at 0x7f87d05ccd90>,
 'pass 2': <src.utils.modeling.ForecastingModel at 0x7f878d5f4250>,
 'pass 3': <src.utils.modeling.ForecastingModel at 0x7f878d602340>,
 'full': <src.utils.modeling.ForecastingModel at 0x7f878d60e430>}

In [5]:
model['full'].submodels_

{'DEF0C': <statsmodels.tsa.holtwinters.HoltWintersResultsWrapper at 0x7f878d60e460>,
 'DE111': <statsmodels.tsa.holtwinters.HoltWintersResultsWrapper at 0x7f878d3544c0>}

In [6]:
modeling = {
    'targets': ['DEF0C', 'DE111']
}

In [7]:
cv_splits_positions = pickle.load(open('../data/05_model_input/splits_positions.pkl/2020-10-08T15.33.52.010Z/splits_positions.pkl', 'rb'))
cv_splits_positions

{'pass 1': {'train': slice(0, 542, None), 'val': slice(542, 549, None)},
 'pass 2': {'train': slice(0, 719, None), 'val': slice(719, 726, None)},
 'pass 3': {'train': slice(0, 896, None), 'val': slice(896, 903, None)}}

In [8]:
df_infer = capacity_factors_daily_2000to2015 = pd.read_hdf(
    path_or_buf='../data/05_model_input/df_infer.hdf', 
    key='df_infer'
)

df_test = capacity_factors_daily_2000to2015 = pd.read_hdf(
    path_or_buf='../data/05_model_input/df_test.hdf', 
    key='df_test'
)

display(df_infer.head(3), df_infer.shape, df_test.head(3), df_test.shape)

,DE145,DE114,DE146,DE132,DE12A,DE133,DE12C,DE11C,DE118,DE119,...,DEG01,DEG0F,DE275,DE21C,DE234,DE251,DE276,DE278,DE718,DE943
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,0.219507,0.269458,0.240540,0.263019,0.593148,0.331149,0.555176,0.284140,0.357407,0.410291,...,0.300818,0.363388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-02,0.083655,0.063571,0.043303,0.005984,0.080731,0.007986,0.066120,0.083709,0.078740,0.108224,...,0.150699,0.133930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-03,0.246707,0.229298,0.337413,0.047907,0.279092,0.136004,0.237430,0.320095,0.460150,0.516133,...,0.348737,0.297686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(903, 292)

,DE145,DE114,DE146,DE132,DE12A,DE133,DE12C,DE11C,DE118,DE119,...,DEG01,DEG0F,DE275,DE21C,DE234,DE251,DE276,DE278,DE718,DE943
date,,,,,,,,,,,,,,,,,,,,,
2015-06-23,0.366372,0.282144,0.169123,0.102384,0.299887,0.085913,0.264382,0.337191,0.124571,0.252602,...,0.134666,0.162422,0.573021,0.416591,0.663477,0.429502,0.499571,0.522793,0.507254,0.238118
2015-06-24,0.011242,0.008288,0.012873,0.006662,0.006408,0.011105,0.008853,0.012184,0.006574,0.024096,...,0.068389,0.070911,0.063352,0.037593,0.179141,0.085772,0.045303,0.043444,0.097510,0.248150
2015-06-25,0.007218,0.007427,0.004105,0.006482,0.011570,0.007547,0.008719,0.007859,0.003046,0.003604,...,0.002606,0.002760,0.002339,0.003077,0.007730,0.002873,0.003176,0.003211,0.021591,0.055783


(7, 292)

In [10]:
import sys
sys.path.append('..')
scaler = pickle.load(open('../data/05_model_input/scaler.pkl/2020-10-09T15.53.34.176Z/scaler.pkl', 'rb'))
scaler

Scaler(steps=[('quantiletransformer',
               QuantileTransformer(output_distribution='normal',
                                   random_state=0)),
              ('makestrictlypositive', MakeStrictlyPositive())])

# Making Predictions

In [11]:
targets = modeling['targets']

In [17]:
preds = {}
for pass_id in cv_splits_positions.keys():
    preds[pass_id] = {}
    for cat in ['train', 'val']:
        yhat = {}
        
        window = df_infer[ cv_splits_positions[pass_id][cat]].index
        start = window[0]
        end = window[-1]

        preds[pass_id][cat] = model[pass_id].predict(start, end, scaler) 
        
# model trained on entire inference dataset
preds['full'] = {}

for cat in ['train', 'test']:
    
    if cat == 'train':
        df = df_infer
    else: 
        df = df_test
        
    start = df.index[0]
    end = df.index[-1]
        
    preds['full'][cat] = model['full'].predict(start, end, scaler) 

In [18]:
preds['full']['test']

,DEF0C,DE111
2015-06-23,0.102816,0.063786
2015-06-24,0.098686,0.066355
2015-06-25,0.099991,0.058471
2015-06-26,0.110583,0.059606
2015-06-27,0.113829,0.067134
2015-06-28,0.131787,0.071151
2015-06-29,0.119016,0.060346


In [30]:
gtruth = {}
preds = {}
targets = model['full'].modeling['targets']
for pass_id in cv_splits_positions.keys():
    gtruth[pass_id] = {}
    preds[pass_id] = {}
    for cat in ['train', 'val']:
        y = {}
        yhat = {}
        
        window = df_infer[ cv_splits_positions[pass_id][cat]].index
        start = window[0]
        end = window[-1]
        
        gtruth[pass_id][cat] = df_infer[slice(start, end)]
        preds[pass_id][cat] = model[pass_id].predict(start, end, scaler) 
        
# model trained on entire inference dataset
gtruth['full'] = {}
preds['full'] = {}

for cat in ['train', 'test']:
    
    if cat == 'train':
        df = df_infer
    else: 
        df = df_test
        
    start = df.index[0]
    end = df.index[-1]
    
    gtruth['full'][cat] = df[slice(start, end)][targets]
    preds['full'][cat] = model['full'].predict(start, end, scaler) 

In [31]:
gtruth['full'][cat]

,DEF0C,DE111
date,,
2015-06-23,0.133906,0.141192
2015-06-24,0.329115,0.007049
2015-06-25,0.043115,0.004975
2015-06-26,0.011318,0.004748
2015-06-27,0.215408,0.044574
2015-06-28,0.125797,0.005654
2015-06-29,0.102188,0.007407


In [32]:
preds['full'][cat]

,DEF0C,DE111
2015-06-23,0.102816,0.063786
2015-06-24,0.098686,0.066355
2015-06-25,0.099991,0.058471
2015-06-26,0.110583,0.059606
2015-06-27,0.113829,0.067134
2015-06-28,0.131787,0.071151
2015-06-29,0.119016,0.060346


# Evaluating

In [ ]:
mean_squared_error(y, y_hat, multioutput='raw_values') ** 0.5